# HỌC MÁY - BÀI THỰC HÀNH 2

In [10]:
import pandas as pd

# Đọc dữ liệu từ file CSV
train_data  = pd.read_csv('Buoi 2/train.csv')
test_data = pd.read_csv('Buoi 2/test.csv')
print(train_data.head())

test_data.head()

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## Các phương pháp tiền xử lí


In [13]:
# Import các thư viện cần thiết
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler

# Điền giá trị thiếu cho cột Age bằng giá trị trung bình
train_data['Age'].fillna(train_data['Age'].mean(), inplace=True)
train_data['Embarked'].fillna(train_data['Embarked'].mode()[0], inplace=True)

# Điền giá trị thiếu cho cột Embarked bằng giá trị phổ biến nhất
test_data['Age'].fillna(test_data['Age'].mean(), inplace=True)
test_data['Embarked'].fillna(test_data['Embarked'].mode()[0], inplace=True)
test_data.drop(columns=['Cabin'])



# Điền giá trị thiếu cho cột Fare trong tập test bằng giá trị trung bình
test_data['Fare'].fillna(test_data['Fare'].mean(), inplace=True)



# One-hot encoding cho các cột categorical
categorical_features = ['Pclass', 'Sex', 'Embarked']
train_encoded = pd.get_dummies(train_data, columns=categorical_features)
test_encoded = pd.get_dummies(test_data, columns=categorical_features)

# Đảm bảo các cột của tập train và test đồng nhất
train_encoded, test_encoded = train_encoded.align(test_encoded, join='left', axis=1, fill_value=0)

### 1. Phương pháp Min-Max Scaling

In [12]:
# Chọn các cột cần chuẩn hóa
numerical_features = ['Age', 'Fare', 'SibSp', 'Parch']

# Áp dụng Min-Max Scaling
minmax_scaler = MinMaxScaler()
train_encoded_minmax = train_encoded.copy()
test_encoded_minmax = test_encoded.copy()
train_encoded_minmax[numerical_features] = minmax_scaler.fit_transform(train_encoded_minmax[numerical_features])
test_encoded_minmax[numerical_features] = minmax_scaler.transform(test_encoded_minmax[numerical_features])

# Tách dữ liệu train thành train và validation
X = train_encoded_minmax.drop(['Survived', 'PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
y = train_encoded_minmax['Survived']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Huấn luyện Decision Tree
clf_minmax = DecisionTreeClassifier(random_state=42)
clf_minmax.fit(X_train, y_train)

# Dự đoán trên tập validation
y_pred_minmax = clf_minmax.predict(X_val)

# Tính các độ đo
accuracy_minmax = accuracy_score(y_val, y_pred_minmax)
precision_minmax = precision_score(y_val, y_pred_minmax)
recall_minmax = recall_score(y_val, y_pred_minmax)
f1_minmax = f1_score(y_val, y_pred_minmax)

print(f'Min-Max Scaling - Accuracy: {accuracy_minmax:.4f}')
print(f'Min-Max Scaling - Precision: {precision_minmax:.4f}')
print(f'Min-Max Scaling - Recall: {recall_minmax:.4f}')
print(f'Min-Max Scaling - F1 Score: {f1_minmax:.4f}')


Min-Max Scaling - Accuracy: 0.7877
Min-Max Scaling - Precision: 0.7368
Min-Max Scaling - Recall: 0.7568
Min-Max Scaling - F1 Score: 0.7467


## Lý do sử dụng:
### Min-Max Scaling: Đảm bảo dữ liệu đồng nhất

### 2. Phương pháp Standardization

In [14]:
# Áp dụng Standardization
standard_scaler = StandardScaler()
train_encoded_standard = train_encoded.copy()
test_encoded_standard = test_encoded.copy()
train_encoded_standard[numerical_features] = standard_scaler.fit_transform(train_encoded_standard[numerical_features])
test_encoded_standard[numerical_features] = standard_scaler.transform(test_encoded_standard[numerical_features])

# Tách dữ liệu train thành train và validation
X = train_encoded_standard.drop(['Survived', 'PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
y = train_encoded_standard['Survived']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Huấn luyện Decision Tree
clf_standard = DecisionTreeClassifier(random_state=42)
clf_standard.fit(X_train, y_train)

# Dự đoán trên tập validation
y_pred_standard = clf_standard.predict(X_val)

# Tính các độ đo
accuracy_standard = accuracy_score(y_val, y_pred_standard)
precision_standard = precision_score(y_val, y_pred_standard)
recall_standard = recall_score(y_val, y_pred_standard)
f1_standard = f1_score(y_val, y_pred_standard)

print(f'Standardization - Accuracy: {accuracy_standard:.4f}')
print(f'Standardization - Precision: {precision_standard:.4f}')
print(f'Standardization - Recall: {recall_standard:.4f}')
print(f'Standardization - F1 Score: {f1_standard:.4f}')


Standardization - Accuracy: 0.7821
Standardization - Precision: 0.7273
Standardization - Recall: 0.7568
Standardization - F1 Score: 0.7417


## Lý do sử dụng:
### Đưa dữ liệu về cùng thang đo.
### Cải thiện tốc độ hội tụ của thuật toán học máy.
### Cải thiện hiệu suất

In [15]:
feature_importances = pd.DataFrame(clf_standard.feature_importances_,
                                   index = X.columns,
                                   columns=['importance']).sort_values('importance', ascending=False)
print(feature_importances)

            importance
Sex_female    0.308977
Fare          0.239912
Age           0.232777
Pclass_3      0.086924
SibSp         0.058832
Parch         0.028086
Pclass_1      0.020307
Embarked_C    0.007746
Embarked_S    0.007410
Embarked_Q    0.007349
Pclass_2      0.001680
Sex_male      0.000000


## Những tri thức rút ra từ bộ dữ liệu
### - Sex: Phụ nữ có tỷ lệ sống sót cao hơn đàn ông.
### - Fare: Mức độ giàu có <=> khả năng tiếp cận các nguồn lực cứu hộ.
### - Age: Trẻ em có tỷ lệ sống sót cao hơn.

## Các đặc trưng ảnh hưởng đến Survived nhất
### - Sex: 0.308977 / 1 
### - Fare:       0.239912 / 1 
### - Age:       0.232777 / 1 

# Kết luận
### Accuracy tốt nhất: 0.7821 - Phương pháp Standardization - Mô hình Cây quyết định
### Precision tốt nhất: 0.7368 - Phương pháp Min-Max Scaling - Mô hình Cây quyết định
### Recall tốt nhất: 0.7568 - Ngang nhau - Mô hình Cây quyết định
### F1 Score tốt nhất: 0.7467 - Phương pháp Min-Max Scaling - Mô hình Cây quyết định